In [2]:
from itertools import cycle
import numpy as np
from scipy.integrate import solve_ivp
from ipywidgets import Button, VBox, HTMLMath

import plotlymath as pm
from myutils import interact, latex


In [3]:
pm.set_defaults(margins=(40))
colors = pm.plotly.colors.DEFAULT_PLOTLY_COLORS

In [4]:
def stability_text(jacobian):
    trace = np.trace(jacobian)
    det = np.linalg.det(jacobian)
    if det < 0:
        return "Saddle point"
    if trace == 0 or det == 0:
        return "Unknown"
    if trace**2 - 4*det < 0:
        return "Stable spiral" if trace < 0 else "Unstable spiral"
    return "Sink" if trace < 0 else "Source"

def stability_color(jacobian):
    trace = np.trace(jacobian)
    det = np.linalg.det(jacobian)
    if det < 0:
        return "darkorange"
    if trace == 0 or det == 0:
        return "gray"
    return "darkgreen" if trace < 0 else "darkred"

def clickable_phase_portrait(f, xrange, yrange, tmax, **options):
    axes_labels = options.get("axes_labels", (r"$x$", r"$y$"))
    fixed_points = options.get("fixed_points", ())
    jacobian = options.get("jacobian", None)
    fixed_points_color = [stability_color(jacobian(0, pt)) 
            for pt in fixed_points] if jacobian else "black"
    fixed_points_color = options.get("fixed_points_color", fixed_points_color)
    fixed_points_text = [stability_text(jacobian(0, pt)) 
            for pt in fixed_points] if jacobian else ""
    vector_field_color = options.get("vector_field_color", "limegreen")
    vector_field_opacity = options.get("vector_field_opacity", 0.6)
    colors = options.get("colors", pm.plotly.colors.DEFAULT_PLOTLY_COLORS)

    figure, plot = pm.make_figure(widget=True)
    figure.layout.hovermode = "closest"
    figure.layout.update(width=750, height=500)
    plot.axes_labels(*axes_labels)
    plot.axes_ranges(xrange, yrange)
    xmin, xmax = xrange
    ymin, ymax = yrange
    x = np.linspace(xmin, xmax, 51)
    y = np.linspace(ymin, ymax, 51)
    xy = np.moveaxis(np.meshgrid(x, y), 0, 2).reshape(-1, 2)
    plot.points(xy, size=15, opacity=0, hoverinfo="none", showlegend=False, id="grid")

    if vector_field_color:
        plot.vector_field(lambda x, y: f(0, (x, y)), xrange, yrange, 
                color=vector_field_color, opacity=vector_field_opacity, 
                name="Vector field", visible="legendonly", hoverinfo="skip")
    if fixed_points:
        plot.points(fixed_points, color=fixed_points_color, size=10, 
                name="Fixed points", visible="legendonly", hoverinfo="x+y+text", 
                hovertext=fixed_points_text)

    color = cycle(colors)
    options = dict(hoverinfo="skip", showlegend=False, id="solutions[]")
    solve_options = dict(method="RK45", dense_output=True)
    #if jacobian:
    #    solve_options["jac"] = jacobian
    def click_handler(trace, points, state):
        if not (points.xs and points.ys):
            return
        initial_state = (points.xs[0], points.ys[0])
        solution = solve_ivp(f, (0, tmax), initial_state, **solve_options)
        plot.parametric(solution.sol, (0, tmax), color=next(color), **options)
    plot["grid"].on_click(click_handler)

    def clear_figure(widget):
        nonlocal color
        color = cycle(colors)
        with figure.batch_update():
            try:
                del plot["solutions[]"]
            except KeyError:
                pass
    clear_button = Button(description="Clear")
    clear_button.on_click(clear_figure)

    return VBox((clear_button, figure))


In [9]:
def linear_phase_portrait(A):
    def f(t, state): return A @ state
    def jacobian(t, state): return A
    vbox = clickable_phase_portrait(f, (-10, 10), (-10, 10), 50, jacobian=jacobian)
    figure = vbox.children[1]
    plot = pm.PlotlyAxes(figure, {}, 1, 1)
    plot.axes_ranges((-10, 10), (-10, 10), scale=(1, 1))
    evalues, S = np.linalg.eig(A)
    if evalues[0].imag == 0:
        for evalue, evector, show in zip(evalues, S.transpose(), (True, False)):
            color = "darkgreen" if evalue < 0 else "darkred" if evalue > 0 else "gray"
            plot.lines((-15*evector, 15*evector), color=color, line_width=3, 
                    opacity=0.5, hoverinfo="skip", name="Eigenvectors", 
                    visible="legendonly", showlegend=show, legendgroup="evecs")
            plot.points((9*evector,), mode="text", text=fr"$\lambda = {evalue:.2f}$", 
                    textfont_size=14, hoverinfo="skip", 
                    visible="legendonly", showlegend=False, legendgroup="evecs")
    else:
        evector = S[:,0]
        # Choose an eigenvector whose real and imag parts are perpendicular
        z = evector @ evector
        if z.imag != 0: # If this is 0, they already are perpendicular
            evector = S[:,1] * (z + abs(z)) / abs(z + abs(z))
        evec1, evec2 = evector.real, evector.imag
        evalue = round(evalues[0].real, 8)
        color = "darkgreen" if evalue < 0 else "darkred" if evalue > 0 else "gray"
        evalue = fr"$\lambda = {latex(evalues[0], round=2, conjpair=True)}$"
        plot.lines((8*evec1, 0*evec1, 8*evec2), color=color, line_width=3, 
                opacity=0.5, hoverinfo="skip", name="Eigenvectors", 
                visible="legendonly", legendgroup="evecs")
        plot.points((9*evec2,), mode="text", text=evalue, 
                textfont_size=14, hoverinfo="skip", 
                visible="legendonly", showlegend=False, legendgroup="evecs")
    return vbox


# The function `linear_phase_portrait`

You can pass this function any $2 \times 2$ matrix $A$, and it will show you the vector field of the differential equation 
$$ \vec{x}\,' = A \vec{x} $$
You can click anywhere in the state space, and it will plot the solution of this differential equation that starts from that point. 

It will also show the eigenlines and eigenvalues of $A$. (If the eigenvalues are non-real, it will show the real and imaginary parts of the corresponding eigenvector, which form the major and minor radii of the ellipse that the solutions rotate around.) Note that you can toggle on/off the display of the vector field and the eigenlines by clicking their labels in the legend. 


In [12]:
A = np.array((
    ( 1,  3), 
    (-2,  0), 
))
linear_phase_portrait(A)

    'data': [{'hoverinfo': 'no…

In [62]:
display(HTMLMath(latex(A)))
evalues, S = np.linalg.eig(A)
for evalue, evector in zip(evalues, S.transpose()):
    evalue = latex(evalue, round=2)
    evector = latex(evector, round=2)
    display(HTMLMath(f"Eigenvalue ${evalue}$ with eigenvector ${evector}$"))


HTMLMath(value='\\begin{pmatrix} -1 & 3 \\\\ -2 & 0 \\end{pmatrix}')

HTMLMath(value='Eigenvalue $-0.5 + 2.4i$ with eigenvector $\\begin{bmatrix} 0.77 \\\\ 0.13 + 0.62i \\end{bmatr…

HTMLMath(value='Eigenvalue $-0.5 - 2.4i$ with eigenvector $\\begin{bmatrix} 0.77 \\\\ 0.13 - 0.62i \\end{bmatr…